In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
import time
import re
import os
import datetime
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
#options.add_argument('-headless')
driverPath='D:/github/1092/python_web/20210525/chromedriver_win32/chromedriver.exe'
url = "https://nidss.cdc.gov.tw/nndss/DiseaseMap?id=19CoV"
browser = webdriver.Chrome(executable_path=driverPath,options=options)
browser.get(url)
time.sleep(2)
County_Name = []
repeat = 0
Today = (datetime.date.today()).strftime("%Y-%m-%d")
Yesterday = (datetime.date.today() + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
#create the new file
#Town total cases
tt_Path = r"D:\github\1092\python_web\Final Project\cases\Town_Total"
TT_Path = tt_Path+  "/" + Today + "-Taiwan town's total cases" + ".csv"
TT_f = open(TT_Path, "w+")  
TT_f.write("TT_county"+","+"TT_town"+","+"TT_cases"+","+"TT_repeat"+"\n")
for d in range(1,7):
    D = browser.find_elements_by_xpath("//div[@class='row row-map']/button")
    action = webdriver.common.action_chains.ActionChains(browser)
    action.move_to_element_with_offset(D[d],5,5)
    action.click()
    action.perform()
    time.sleep(2)
    #for length
    P = browser.find_elements_by_xpath("//div[@class='highcharts-container ']/*[name()='svg']/*[name()='g']/*[name()='g']")            
            
    for p in range(len(P)):
    #   get county's <g> tag--->included not county related <g> tag
        P = browser.find_elements_by_xpath("//div[@class='highcharts-container ']/*[name()='svg']/*[name()='g']/*[name()='g']")        
        #filter out county related <g> tag
        if re.findall("highcharts-label highcharts-data-label highcharts-data-label-color-(\d) highcharts-drilldown-data-label(.*)",str(P[p].get_attribute('class'))):
#             print("**************************************************************************")
            county = P[p].find_elements_by_tag_name('tspan')[0].text
#             print("**************************************************************************")
            # Click on County to get town cases data
            #scroll the window let all towns can be clicked
            browser.execute_script("window.scrollTo(0,250)") 
            P[p].click()
            time.sleep(3)

            #extend the comfirmed table menu(can't get the table directly by scrap web element---->will have losing data--->need to click extend table first)
            extend = browser.find_element_by_xpath("//img[@class='tableMenu']")
            extend.click()
            time.sleep(1)
            
            town_table=browser.find_elements_by_xpath("//table[@class = 'dataTables1 dataTable']//tbody//tr")
            #get District name(indluing 其他) and confirmed number
            for tt in town_table:
                data = county + "," + tt.text.replace(" ",",")+"\n"
#                 print(data)
                #filter out town name
                prepro_data = re.sub("\d","",data)
                prepro_data = re.sub(",","",prepro_data)
                prepro_data = re.sub("\n","",prepro_data)
                prepro_data = prepro_data[3:]
                #avoid repeat town name
                trigger = 0
                for CN in County_Name:
                    if CN == prepro_data:
                        trigger = trigger + 1
                    else:
                        pass
                if trigger == 0:
                    County_Name.append(prepro_data)
                    TT_f.write(data)
                else:
                    County_Name.append(prepro_data)
                    data = county + "," + tt.text.replace(" ",",")+","+str(trigger)+"\n"
                    repeat =repeat + 1
                    TT_f.write(data)
                    

            Back = browser.find_elements_by_xpath("//div[@class='highcharts-container ']/*[name()='svg']/*[name()='g']")
            action = webdriver.common.action_chains.ActionChains(browser)
            action.move_to_element_with_offset(Back[6], 0.1,0.1 )
            action.click()
            action.perform()

            time.sleep(1)
            browser.refresh()
            time.sleep(2)
        else:
            pass
TT_f.close()
browser.close()


#import today and yesterday town total cases
today = pd.read_csv(tt_Path +  "/" + Today + "-Taiwan town's total cases" + ".csv",encoding = 'big5')
yesterday = pd.read_csv(tt_Path +  "/" + Yesterday + "-Taiwan town's total cases" + ".csv",encoding = 'big5')

#Town Daily cases
td_Path = r"D:\github\1092\python_web\Final Project\cases\Town_Daily"
TD_Path = td_Path +  "/" + Today + "-Taiwan town's Daily cases" + ".csv"
TD_f = open(TD_Path, "w+")  
TD_f.write("TD_county"+","+"TD_town"+","+"TD_cases"+","+"TD_repeat"+"\n")
if len(today)==len(yesterday):
    for d in range(len(today)):
        TD_f.write(today['TT_county'][d]+","+today['TT_town'][d]+","+str(int(today['TT_cases'][d]-yesterday['TT_cases'][d]))+",")
        if str(today['TT_repeat'][d]) =='nan':
            TD_f.write(""+"\n")
        else:
            TD_f.write(str(today['TT_repeat'][d])+"\n")
        
#         print(today['TT_county'][d]+","+today['TT_town'][d]+","+str(today['TT_cases'][d]-yesterday['TT_cases'][d])+","+str(today['TT_repeat'][d])+"\n")
else:
    print("Error data length")
TD_f.close()

# print("*********************************************************************************************")
#County_total&County_daily
cd_Path = r"D:\github\1092\python_web\Final Project\cases\County_Daily"
CD_Path = cd_Path +  "/" + Today + "-Taiwan County's Daily cases" + ".csv"
CD_f = open(CD_Path, "w+")  
CD_f.write("CD_county"+","+"CD_cases"+"\n")

ct_Path = r"D:\github\1092\python_web\Final Project\cases\County_Total"
CT_Path = ct_Path +  "/" + Today + "-Taiwan County's total cases" + ".csv"
CT_f = open(CT_Path, "w+")  
CT_f.write("CT_county"+","+"CT_cases"+"\n")

TNumber = 0
YNumber = 0
for i in range(len(today)):
    if i > 0:
        if today.iloc[i]['TT_county'] == yesterday.iloc[i]['TT_county'] == TName:
            TNumber = TNumber+today.iloc[i]['TT_cases']
            YNumber = YNumber+yesterday.iloc[i]['TT_cases']
            
        else:
#             print(TName + " " + str(TNumber))
#             print(YName + " " + str(YNumber))
#             print(YName + " " + str(TNumber - YNumber))
            CT_f.write(TName + "," + str(TNumber)+"\n")
            CD_f.write(YName + "," + str(TNumber - YNumber)+"\n")
            TName = today.iloc[i]['TT_county']
            YName = yesterday.iloc[i]['TT_county']
            TNumber = YNumber = 0
            TNumber = TNumber+today.iloc[i]['TT_cases']
            YNumber = YNumber+yesterday.iloc[i]['TT_cases']
    else:
        TName = today.iloc[i]['TT_county']
        YName = yesterday.iloc[i]['TT_county']
        TNumber = TNumber+today.iloc[i]['TT_cases']
        YNumber = YNumber+yesterday.iloc[i]['TT_cases']
#最後一個縣市
CD_f.write(YName + "," + str(TNumber - YNumber)+"\n")
CT_f.write(TName + "," + str(TNumber)+"\n")

CD_f.close()    
CT_f.close()


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'